## How does bad history influence the distribution

In [ ]:
from hydra import compose, initialize
from omegaconf import OmegaConf
initialize(config_path="../../configs", version_base="1.3")
cfg = compose(config_name="eval_sim")

In [ ]:
from open_anything_diffusion.models.modules.dit_models import DiT
from open_anything_diffusion.models.modules.history_encoder import HistoryEncoder
from open_anything_diffusion.models.flow_diffuser_hisdit import (
    FlowTrajectoryDiffuserSimulationModule_HisDiT,
)
network = {
    "DiT": DiT(
        in_channels=3 + 3 + 128,
        depth=5,
        hidden_size=128,
        num_heads=4,
        learn_sigma=True,
    ).cuda(),
    "History": HistoryEncoder(
        history_dim=128, history_len=1, batch_norm=False
    ).cuda(),
}

# ckpt_file = "/home/yishu/open_anything_diffusion/logs/train_trajectory_diffuser_hisdit/2024-05-01/10-18-38/checkpoints/epoch=529-step=293090-val_loss=0.00-weights-only.ckpt"
ckpt_file = "/home/yishu/open_anything_diffusion/logs/train_trajectory_diffuser_hisdit/2024-05-10/12-09-08/checkpoints/epoch=439-step=243320-val_loss=0.00-weights-only.ckpt"
history_model = FlowTrajectoryDiffuserSimulationModule_HisDiT(
    network, inference_cfg=cfg.inference, model_cfg=cfg.model
).cuda()
history_model.load_from_ckpt(ckpt_file)
history_model.eval()

In [ ]:
obj_id = "9065"
import os
from rpad.partnet_mobility_utils.data import PMObject
pm_dir = os.path.expanduser("~/datasets/partnet-mobility/convex")
# env = PMSuctionSim(obj_id, pm_dir, gui=gui)
raw_data = PMObject(os.path.join(pm_dir, obj_id))

In [ ]:
from open_anything_diffusion.simulations.suction import GTFlowModel, PMSuctionSim
env = PMSuctionSim(obj_id, pm_dir, gui=False)
gt_model = GTFlowModel(raw_data, env)

env.reset()

available_joints = raw_data.semantics.by_type(
    "hinge"
) + raw_data.semantics.by_type("slider")
print(available_joints)

target_link = available_joints[0].name
import pybullet as p
info = p.getJointInfo(
    env.render_env.obj_id,
    env.render_env.link_name_to_index[target_link],
    env.render_env.client_id,
)
init_angle, target_angle = info[8], info[9]
print(init_angle, target_angle)

### Set state

In [ ]:
import numpy as np

# Fully closed
env.set_joint_state(target_link, init_angle + np.linspace(0, 1, 20)[0] * (target_angle - init_angle))
# # Open
# env.set_joint_state(target_link, init_angle + np.linspace(0, 1, 20)[10] * (target_angle - init_angle))

pc_obs = env.render(filter_nonobj_pts=True, n_pts=1200)
rgb, depth, seg, P_cam, P_world, pc_seg, segmap = pc_obs

#### 1. No history

In [ ]:
import torch
from open_anything_diffusion.metrics.trajectory import flow_metrics
import tqdm
import copy
trial_cnts = 100
cos_distribution = []
metrics = []
predictions = {}

gt_flow = gt_model(pc_obs)
nonzero_gt_flowixs = torch.where(gt_flow.norm(dim=-1) != 0.0)
gt_flow_nz = gt_flow[nonzero_gt_flowixs]

for i in tqdm.tqdm(range(trial_cnts)):
    with torch.no_grad():
        pred_flow = history_model(copy.deepcopy(pc_obs))[:, 0, :]

    pred_flow_nz = pred_flow[nonzero_gt_flowixs]

    cos_dist = torch.cosine_similarity(pred_flow_nz, gt_flow_nz, dim=-1).mean()
    cos_distribution.append(cos_dist)
    # predictions[cos_dist] = pred_flow

    rmse, cos_dist, mag_error = flow_metrics(pred_flow_nz, gt_flow_nz)
    metrics.append(rmse)
    predictions[rmse] = pred_flow


In [ ]:
import matplotlib.pyplot as plt
plt.scatter([0] * len(metrics), metrics)
plt.ylabel("RMSE")

In [ ]:
plt.scatter([0] * len(cos_distribution), cos_distribution)
plt.ylabel("Cosine")

In [ ]:
no_history_ratio = sum([1 if metric.item() < 0.5 else 0 for metric in metrics])/100
no_history_ratio_cosine = sum([1 if cosine.item() > 0.5 else 0 for cosine in cos_distribution])/100

#### 2. With history

In [ ]:
trial_cnts = 20
with_history_metrics = {}
with_history_cosines = {}

for (history_metric, history_flow), history_cos in tqdm.tqdm(zip(predictions.items(), cos_distribution)):
    with_history_metrics[history_metric] = []
    with_history_cosines[history_cos] = []
    for i in range(trial_cnts):
        with torch.no_grad():
            pred_flow = history_model(copy.deepcopy(pc_obs), copy.deepcopy(pc_obs[4]), history_flow.cpu().numpy())[:, 0, :]

        pred_flow_nz = pred_flow[nonzero_gt_flowixs]

        cos_dist = torch.cosine_similarity(pred_flow_nz, gt_flow_nz, dim=-1).mean()
        with_history_cosines[history_cos].append(cos_dist)

        rmse, cos_dist, mag_error = flow_metrics(pred_flow_nz, gt_flow_nz)
        with_history_metrics[history_metric].append(rmse)


RMSE

In [ ]:
good_history_ratio = 0
good_history_cnt = 0
bad_history_ratio = 0
bad_history_cnt = 0
for history_metric in with_history_metrics.keys():
    if history_metric < 0.5:
        good_history_cnt += 20 
        good_history_ratio += sum(with_history_metrics[history_metric])
    else:
        bad_history_cnt += 20
        bad_history_ratio += sum(with_history_metrics[history_metric])

print("Mean(rmse) with good history: ", good_history_ratio / good_history_cnt)
print("Mean(rmse) with bad history: ", bad_history_ratio / bad_history_cnt)

In [ ]:
import matplotlib.pyplot as plt

xs = []
ys = []
ratios = []
for history_cos in with_history_metrics.keys():
    xs += [history_cos.item()] * 20
    ys += with_history_metrics[history_cos]
    ratios.append(sum([1 if cosine.item() < 0.5 else 0 for cosine in with_history_metrics[history_cos]])/20)
plt.scatter(xs, ys)
plt.xlabel('History rmse')
plt.ylabel('This step rmse')

In [ ]:
plt.scatter(list(with_history_metrics.keys()), ratios)
plt.hlines(no_history_ratio, min(list(with_history_metrics.keys())), max(list(with_history_metrics.keys())))
plt.xlabel('History rmse')
plt.ylabel('This step rmse < 0.5 ratio')

Cosine

In [ ]:
good_history_ratio = 0
good_history_cnt = 0
bad_history_ratio = 0
bad_history_cnt = 0
for history_cos in with_history_cosines.keys():
    if history_cos > 0.:
        good_history_cnt += 20 
        good_history_ratio += sum(with_history_cosines[history_cos])
    else:
        bad_history_cnt += 20
        bad_history_ratio += sum(with_history_cosines[history_cos])

print("Mean(cosine) with good history: ", good_history_ratio / good_history_cnt)
print("Mean(cosine) with bad history: ", bad_history_ratio / bad_history_cnt)
    

In [ ]:
import matplotlib.pyplot as plt

xs = []
ys = []
ratios = []
for history_cos in with_history_cosines.keys():
    xs += [history_cos.item()] * 20
    ys += with_history_cosines[history_cos]
    ratios.append(sum([1 if cosine.item() > 0.5 else 0 for cosine in with_history_cosines[history_cos]])/20)
plt.scatter(xs, ys)
plt.xlabel('History cosine')
plt.ylabel('This step cosine')

In [ ]:
plt.scatter(list(with_history_cosines.keys()), ratios)
plt.hlines(no_history_ratio_cosine, min(list(with_history_cosines.keys())), max(list(with_history_cosines.keys())))
plt.xlabel('History cosine')
plt.ylabel('This step cosine > 0.5 ratio')

### View the best prediction

In [ ]:
predictions[min(list(with_history_metrics.keys()))].shape

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
animation = FlowNetAnimation()
animation.add_trace(
    torch.as_tensor(P_world),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([P_world]),
    torch.as_tensor([predictions[min(list(with_history_metrics.keys()))].cpu().numpy()]),
    "red",
)
fig = animation.animate()
fig.show()